In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
import efficientnet.keras as effnet

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import os
import zipfile
import matplotlib.pyplot as plt
import shutil
import numpy as np
import pandas as pd
import cv2 as cv
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB6
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow import keras
AUTO = tf.data.experimental.AUTOTUNE

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [5]:
! pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"azkaprivanna","key":"841300ca256a5fda39dbb15657efb0a9"}'}

In [6]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets download shubhamgoel27/dermnet
!unzip -q /content/dermnet.zip \
  && rm dermnet.zip

Dataset URL: https://www.kaggle.com/datasets/shubhamgoel27/dermnet
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
100% 1.72G/1.72G [00:27<00:00, 127MB/s]
100% 1.72G/1.72G [00:27<00:00, 68.1MB/s]


In [8]:
train_dir_path = ['/content/train/Acne and Rosacea Photos', '/content/train/Bullous Disease Photos', '/content/train/Exanthems and Drug Eruptions',
            '/content/train/Hair Loss Photos Alopecia and other Hair Diseases', '/content/train/Herpes HPV and other STDs Photos',
            '/content/train/Melanoma Skin Cancer Nevi and Moles', '/content/train/Psoriasis pictures Lichen Planus and related diseases',
            '/content/train/Systemic Disease', '/content/train/Vascular Tumors', '/content/train/Vasculitis Photos', '/content/train/Urticaria Hives',
            '/content/train/Seborrheic Keratoses and other Benign Tumors', '/content/train/Poison Ivy Photos and other Contact Dermatitis',
            '/content/train/Light Diseases and Disorders of Pigmentation']

In [9]:
test_dir_path2 = ['/content/test/Acne and Rosacea Photos', '/content/test/Bullous Disease Photos', '/content/test/Exanthems and Drug Eruptions',
            '/content/test/Hair Loss Photos Alopecia and other Hair Diseases', '/content/test/Herpes HPV and other STDs Photos',
            '/content/test/Melanoma Skin Cancer Nevi and Moles', '/content/test/Psoriasis pictures Lichen Planus and related diseases',
            '/content/test/Systemic Disease', '/content/test/Vascular Tumors', '/content/test/Vasculitis Photos', '/content/test/Urticaria Hives',
            '/content/test/Seborrheic Keratoses and other Benign Tumors', '/content/test/Poison Ivy Photos and other Contact Dermatitis',
            '/content/test/Light Diseases and Disorders of Pigmentation']

In [10]:
for path in train_dir_path + test_dir_path2:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"Deleted: {path}")
    else:
        print(f"Does not exist: {path}")

Deleted: /content/train/Acne and Rosacea Photos
Deleted: /content/train/Bullous Disease Photos
Deleted: /content/train/Exanthems and Drug Eruptions
Deleted: /content/train/Hair Loss Photos Alopecia and other Hair Diseases
Deleted: /content/train/Herpes HPV and other STDs Photos
Deleted: /content/train/Melanoma Skin Cancer Nevi and Moles
Deleted: /content/train/Psoriasis pictures Lichen Planus and related diseases
Deleted: /content/train/Systemic Disease
Deleted: /content/train/Vascular Tumors
Deleted: /content/train/Vasculitis Photos
Deleted: /content/train/Urticaria Hives
Deleted: /content/train/Seborrheic Keratoses and other Benign Tumors
Deleted: /content/train/Poison Ivy Photos and other Contact Dermatitis
Deleted: /content/train/Light Diseases and Disorders of Pigmentation
Deleted: /content/test/Acne and Rosacea Photos
Deleted: /content/test/Bullous Disease Photos
Deleted: /content/test/Exanthems and Drug Eruptions
Deleted: /content/test/Hair Loss Photos Alopecia and other Hair Di

In [11]:
from glob import glob
train_images = glob('/content/train/*/*.jpg')
len(train_images)

7438

In [12]:
images = [path.replace('\\', '/') for path in train_images]
df = pd.DataFrame({'filepath': images})
df['label'] = df['filepath'].str.split('/', expand=True)[3]

In [ ]:
print(df.head())
print(df.tail())

                                            filepath  \
0  /content/train/Lupus and other Connective Tiss...   
1  /content/train/Lupus and other Connective Tiss...   
2  /content/train/Lupus and other Connective Tiss...   
3  /content/train/Lupus and other Connective Tiss...   
4  /content/train/Lupus and other Connective Tiss...   

                                        label  
0  Lupus and other Connective Tissue diseases  
1  Lupus and other Connective Tissue diseases  
2  Lupus and other Connective Tissue diseases  
3  Lupus and other Connective Tissue diseases  
4  Lupus and other Connective Tissue diseases  
                                               filepath  \
7433  /content/train/Warts Molluscum and other Viral...   
7434  /content/train/Warts Molluscum and other Viral...   
7435  /content/train/Warts Molluscum and other Viral...   
7436  /content/train/Warts Molluscum and other Viral...   
7437  /content/train/Warts Molluscum and other Viral...   

                    

In [13]:
label_mapping = {
    'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions':0,
    'Atopic Dermatitis Photos':1,
    'Cellulitis Impetigo and other Bacterial Infections':2,
    'Eczema Photos':3,
    # 'Light Diseases and Disorders of Pigmentation': 4,
    'Lupus and other Connective Tissue diseases': 4,
    'Nail Fungus and other Nail Disease': 5,
    'Scabies Lyme Disease and other Infestations and Bites': 6,
    'Tinea Ringworm Candidiasis and other Fungal Infections': 7,
    'Warts Molluscum and other Viral Infections': 8
}

In [14]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  # Data Augmentation
  train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest',
      brightness_range=[0.8, 1.2]
  )
  # Instantiate the ImageDataGenerator class
  validation_datagen = ImageDataGenerator(
      rescale=1./255
  )

  # Pass in the appropriate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(
      directory=TRAINING_DIR,
      batch_size=16,
      class_mode='categorical',
      target_size=(256, 256),
      shuffle=True)

  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(
      directory=VALIDATION_DIR,
      batch_size=16,
      class_mode='categorical',
      target_size=(256, 256))

  return train_generator, validation_generator

In [15]:
# Define the training and validation base directories
TRAINING_DIR='/content/train'
VALIDATION_DIR='/content/test'

train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

Found 7438 images belonging to 9 classes.
Found 1864 images belonging to 9 classes.


In [16]:
def build_model():
    # Model definition
    efficientnet_model = effnet.EfficientNetB6(
        weights='imagenet',
        include_top=False,
        input_shape=(256, 256, 3)
    )
    efficientnet_model.trainable = False

    # Add custom layers on top of EfficientNet
    # inputs = layers.Input(shape=(256, 256, 3))
    # x = layers.Flatten()(inputs)
    # x = layers.Dense(256, activation='relu')(x)
    # x = layers.BatchNormalization()(x)
    # x = layers.Dense(256, activation='relu')(x)
    # x = layers.Dropout(0.3)(x)
    # x = layers.BatchNormalization()(x)
    # outputs = layers.Dense(len(label_mapping), activation='softmax')(x)

    # Add custom layers on top of EfficientNet
    x = efficientnet_model.output
    x = GlobalAveragePooling2D()(x)
    # x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    # x = Dropout(0.5)(x)
    output = Dense(len(label_mapping), activation='softmax')(x)

    # Combine the base model with custom layers
    # model = Model(inputs, outputs)
    model = Model(inputs=efficientnet_model.input, outputs=output)

    return model

def train_model(model, train_generator, validation_generator):
    # Compile the model
    optimizer = Adam(learning_rate=0.0001)
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer=optimizer,
                  metrics=['accuracy'])

    # early_stopping = EarlyStopping(
    #     monitor='val_loss',
    #     patience=15,
    #     restore_best_weights=True)

    # Train the model
    history = model.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator,
        # callbacks=[early_stopping],
        verbose=2
    )

    return history

In [17]:
# Build and train the model
model = build_model()
history = train_model(model, train_generator, validation_generator)

165527152/165527152 [==============================] - 1s 0us/step
Epoch 1/50
465/465 - 223s - loss: 1.7075 - accuracy: 0.4142 - val_loss: 1.5562 - val_accuracy: 0.4641 - 223s/epoch - 479ms/step
Epoch 2/50
465/465 - 196s - loss: 1.5463 - accuracy: 0.4672 - val_loss: 1.5029 - val_accuracy: 0.4802 - 196s/epoch - 421ms/step
Epoch 3/50
465/465 - 198s - loss: 1.5031 - accuracy: 0.4856 - val_loss: 1.4622 - val_accuracy: 0.5038 - 198s/epoch - 425ms/step
Epoch 4/50
465/465 - 198s - loss: 1.4638 - accuracy: 0.5008 - val_loss: 1.4316 - val_accuracy: 0.5086 - 198s/epoch - 425ms/step
Epoch 5/50
465/465 - 198s - loss: 1.4263 - accuracy: 0.5132 - val_loss: 1.4143 - val_accuracy: 0.5182 - 198s/epoch - 426ms/step
Epoch 6/50
465/465 - 196s - loss: 1.3974 - accuracy: 0.5215 - val_loss: 1.3934 - val_accuracy: 0.5306 - 196s/epoch - 421ms/step
Epoch 7/50
465/465 - 198s - loss: 1.3802 - accuracy: 0.5231 - val_loss: 1.3874 - val_accuracy: 0.5290 - 198s/epoch - 425ms/step
Epoch 8/50
465/465 - 195s - loss: 1.3

In [ ]:
def fine_tune_model(model, train_generator, validation_generator):
    for layer in model.layers[-20:]:
        layer.trainable = True

    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                  metrics=['accuracy'])

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True)

    history_fine = model.fit(
        train_generator,
        epochs=40,
        validation_data=validation_generator,
        callbacks=[early_stopping],
        verbose=2
    )

    return history_fine

# Fine-tune the model
history_fine = fine_tune_model(model, train_generator, validation_generator)

Epoch 1/40
465/465 - 223s - loss: 1.1854 - accuracy: 0.5929 - val_loss: 1.2632 - val_accuracy: 0.5810 - 223s/epoch - 480ms/step
Epoch 2/40
465/465 - 199s - loss: 1.1249 - accuracy: 0.6136 - val_loss: 1.2798 - val_accuracy: 0.5794 - 199s/epoch - 428ms/step
Epoch 3/40
465/465 - 200s - loss: 1.0924 - accuracy: 0.6304 - val_loss: 1.2855 - val_accuracy: 0.5821 - 200s/epoch - 430ms/step
Epoch 4/40
465/465 - 200s - loss: 1.0606 - accuracy: 0.6361 - val_loss: 1.2207 - val_accuracy: 0.5992 - 200s/epoch - 431ms/step
Epoch 5/40
465/465 - 201s - loss: 1.0433 - accuracy: 0.6409 - val_loss: 1.2392 - val_accuracy: 0.5880 - 201s/epoch - 432ms/step
Epoch 6/40
465/465 - 200s - loss: 1.0072 - accuracy: 0.6533 - val_loss: 1.2136 - val_accuracy: 0.5998 - 200s/epoch - 430ms/step
Epoch 7/40
465/465 - 200s - loss: 0.9826 - accuracy: 0.6608 - val_loss: 1.2114 - val_accuracy: 0.6025 - 200s/epoch - 430ms/step
Epoch 8/40
465/465 - 199s - loss: 0.9715 - accuracy: 0.6590 - val_loss: 1.2113 - val_accuracy: 0.5901 - 

In [ ]:
# train_models()
# Save training results
results = pd.DataFrame({
    'Model': ['EfficientNetB2'],
    'Train Accuracy': [np.max(history.history['accuracy'])],
    'Test Accuracy': [np.max(history.history['val_accuracy'])]
})

results_fine = pd.DataFrame({
    'Model': ['EfficientNetB2 (Fine-Tuned)'],
    'Train Accuracy': [np.max(history_fine.history['accuracy'])],
    'Test Accuracy': [np.max(history_fine.history['val_accuracy'])]
})

final_results = pd.concat([results, results_fine], ignore_index=True)
print(final_results)

165234480/165234480 [==============================] - 8s 0us/step
Epoch 1/50
642/642 - 628s - loss: 2.0789 - accuracy: 0.3312 - val_loss: 2.1475 - val_accuracy: 0.3152 - 628s/epoch - 979ms/step
Epoch 2/50
642/642 - 532s - loss: 1.6572 - accuracy: 0.4759 - val_loss: 1.7456 - val_accuracy: 0.4567 - 532s/epoch - 829ms/step
Epoch 3/50
642/642 - 517s - loss: 1.4141 - accuracy: 0.5554 - val_loss: 1.5115 - val_accuracy: 0.5138 - 517s/epoch - 806ms/step
Epoch 4/50
642/642 - 516s - loss: 1.2143 - accuracy: 0.6148 - val_loss: 1.9601 - val_accuracy: 0.3988 - 516s/epoch - 804ms/step
Epoch 5/50


KeyboardInterrupt: 

In [ ]:
from glob import glob

images = glob('/content/train/*/*.jpg')
len(images)

In [ ]:
#replace backslash with forward slash to avoid unexpected errors
import pandas as pd

images = [path.replace('\\', '/') for path in images]
df = pd.DataFrame({'filepath': images})
df['label'] = df['filepath'].str.split('/', expand=True)[3]
df.head()

In [ ]:
df.tail()

In [ ]:
# NKI
Actinic_Keratosis = df['label'].values[df['label'].str.contains('Actinic_Keratosis')].tolist()
Atopic_Dermatitis= df['label'].values[df['label'].str.contains('Atopic_Dermatitis')].tolist()
Cellulitis_Impetigo= df['label'].values[df['label'].str.contains('Cellulitis_Impetigo')].tolist()
Eczema= df['label'].values[df['label'].str.contains('Eczema')].tolist()
Light_Diseases= df['label'].values[df['label'].str.contains('Light_Diseases')].tolist()
Lupus= df['label'].values[df['label'].str.contains('Lupus')].tolist()
Nail_Fungus= df['label'].values[df['label'].str.contains('Nail_Fungus')].tolist()
Poison_Ivy = df['label'].values[df['label'].str.contains('Poison_Ivy')].tolist()
Scabies= df['label'].values[df['label'].str.contains('Scabies')].tolist()
Seborrheic_Keratoses= df['label'].values[df['label'].str.contains('Seborrheic_Keratoses')].tolist()
Tinea_Ringworm= df['label'].values[df['label'].str.contains('Tinea_Ringworm')].tolist()
Urticaria_Hives= df['label'].values[df['label'].str.contains('Urticaria_Hives')].tolist()
Vasculitis= df['label'].values[df['label'].str.contains('Vasculitis')].tolist()
Warts_Molluscum= df['label'].values[df['label'].str.contains('Warts_Molluscum')].tolist()



# Actinic_Keratosis = df['label'].values[df['label'].str.contains('Actinic_Keratosis')].tolist()
# Atopic_Dermatitis = df['label'].values[df['label'].str.contains('Atopic_Dermatitis')].tolist()
# Cellulitis_Impetigo = df['label'].values[df['label'].str.contains('Cellulitis Impetigo')].tolist()
# Eczema = df['label'].values[df['label'].str.contains('Eczema')].tolist()

In [ ]:
# NKI
df_images = {
    'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions':Actinic_Keratosis,
    'Atopic Dermatitis Photos':Atopic_Dermatitis,
    'Cellulitis Impetigo and other Bacterial Infections':Cellulitis_Impetigo,
    'Eczema Photos':Eczema,
    'Light Diseases and Disorders of Pigmentation': Light_Diseases,
    'Lupus and other Connective Tissue diseases': Lupus,
    'Nail Fungus and other Nail Disease': Nail_Fungus,
    'Poison Ivy Photos and other Contact Dermatitis': Poison_Ivy,
    'Scabies Lyme Disease and other Infestations and Bites': Scabies,
    'Seborrheic Keratoses and other Benign Tumors': Seborrheic_Keratoses,
    'Tinea Ringworm Candidiasis and other Fungal Infections': Tinea_Ringworm,
    'Urticaria Hives': Urticaria_Hives,
    'Vasculitis Photos': Vasculitis,
    'Warts Molluscum and other Viral Infections': Warts_Molluscum
}

In [ ]:
df_labels = {
    'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions':0,
    'Atopic Dermatitis Photos':1,
    'Cellulitis Impetigo and other Bacterial Infections':2,
    'Eczema Photos':3,
    'Light Diseases and Disorders of Pigmentation': 4,
    'Lupus and other Connective Tissue diseases': 5,
    'Nail Fungus and other Nail Disease': 6,
    'Poison Ivy Photos and other Contact Dermatitis': 7,
    'Scabies Lyme Disease and other Infestations and Bites': 8,
    'Seborrheic Keratoses and other Benign Tumors': 9,
    'Tinea Ringworm Candidiasis and other Fungal Infections': 10,
    'Urticaria Hives': 11,
    'Vasculitis Photos': 12,
    'Warts Molluscum and other Viral Infections': 13
}

In [ ]:
# NKI
import cv2 as cv
import numpy as np

X = []
y = []

for label, images in df_images.items():
    for image in images:
        img = cv.imread(str(image))
        if img is None:
            print(f"Failed to read image: {image}")
            continue
        img = cv.resize(img, (224, 224))
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        X.append(img)
        y.append(df[label])

# Convert lists to arrays
X = np.array(X)
y = np.array(y)


In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  """
  Creates the training and validation data generators

  Args:
    TRAINING_DIR (string): directory path containing the training images
    VALIDATION_DIR (string): directory path containing the testing/validation images

  Returns:
    train_generator, validation_generator - tuple containing the generators
  """

  # # Instantiate the ImageDataGenerator class (don't forget to set the arguments to augment the images)
  # train_datagen = ImageDataGenerator(rescale=1./255)

  # # Pass in the appropriate arguments to the flow_from_directory method
  # train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
  #                                                     batch_size=128,
  #                                                     class_mode='categorical',
  #                                                     target_size=(224, 224),
  #                                                     shuffle=True)

  # # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  # validation_datagen = ImageDataGenerator(rescale=1./255)

  # # Pass in the appropriate arguments to the flow_from_directory method
  # validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
  #                                                               batch_size=64,
  #                                                               class_mode='categorical',
  #                                                               target_size=(224, 224))
  # Data augmentation
  train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
  )

  test_datagen = ImageDataGenerator(rescale=1./255)

  train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
  validation_datagen = ImageDataGenerator(rescale=1./255)
  validation_generator = test_datagen.flow(X_test, y_test, batch_size=32)



  return train_generator, validation_generator

In [ ]:
# Define the training and validation base directories
TRAINING_DIR='/content/train'
VALIDATION_DIR='/content/test'

train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

### **SIMPLE CNN**

In [ ]:
##SIMPLE CNN
from keras.optimizers import Adam

def create_model():
  '''Creates a CNN with 4 convolutional layers'''
  modelio = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(14, activation='softmax')
  ])

  modelio.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate=1e-4),
                metrics=['accuracy'])

  return modelio

In [ ]:
# Constant for epochs
EPOCHS = 60

# Create a new model
modelin = create_model()

# Train the model
history = modelin.fit(
      train_generator,
      steps_per_epoch=40,  # 2000 images = batch_size * steps
      epochs=EPOCHS,
      validation_data=validation_generator,
      validation_steps=40,  # 1000 images = batch_size * steps
      verbose=2)

In [ ]:
def plot_loss_acc(history):
  '''Plots the training and validation loss and accuracy from a history object'''
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'bo', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Training Loss')
  plt.plot(epochs, val_loss, 'b', label='Validation Loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

In [ ]:
# Plot training results
plot_loss_acc(history)

### **FOR WHAT?**

In [ ]:
#FOR WHAT?
from tensorflow.keras.preprocessing import image_dataset_from_directory
train_dataset = image_dataset_from_directory(TRAINING_DIR,
                                             shuffle=True,
                                             batch_size=256,
                                             image_size=(224,224))

In [ ]:
#FOR WHAT?
from tensorflow.keras.preprocessing import image_dataset_from_directory
test_dataset = image_dataset_from_directory(VALIDATION_DIR,
                                             shuffle=True,
                                             batch_size=64,
                                             image_size=(224,224))

In [ ]:
#FOR WHAT?
class_names = train_dataset.class_names

print(class_names)

In [ ]:
def get_label_from_path(path):
  """Returns the class or directory name from a path."""
  return os.path.basename(path)

path = ['/content/train/Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', '/content/train/Atopic Dermatitis Photos',
        '/content/train/Cellulitis Impetigo and other Bacterial Infections', '/content/train/Eczema Photos',
        '/content/train/Light Diseases and Disorders of Pigmentation', '/content/train/Lupus and other Connective Tissue diseases',
        '/content/train/Lupus and other Connective Tissue diseases', '/content/train/Nail Fungus and other Nail Disease',
        '/content/train/Poison Ivy Photos and other Contact Dermatitis', '/content/train/Scabies Lyme Disease and other Infestations and Bites',
        '/content/train/Seborrheic Keratoses and other Benign Tumors', '/content/train/Tinea Ringworm Candidiasis and other Fungal Infections',
        '/content/train/Urticaria Hives', '/content/train/Vasculitis Photos', '/content/train/Warts Molluscum and other Viral Infections']

for x in path:
  train_label = get_label_from_path(x)
  print(train_label)  # Output: "class_directory"


### **CALLBACK**

In [ ]:
# Define a Callback class that stops training once accuracy reaches 95.0%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      print("\nReached 95.0% accuracy so cancelling training!")
      self.model.stop_training = True

### **EFFICIENTNETB0**

In [ ]:
from keras.applications import EfficientNetB0

pre_trained_model = EfficientNetB0(input_shape = (224, 224, 3),
                                include_top = False,
                                weights = 'imagenet')

# Freeze the weights of the layers.
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
pre_trained_model.summary()

In [ ]:
# Choose `block5c_add` as the last layer of your base model
last_layer = pre_trained_model.get_layer('block5c_add')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(14, activation='softmax')(x)

# Append the dense network to the base model
model = Model(pre_trained_model.input, x)

# Print the model summary. See your dense network connected at the end.
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer = Adam(learning_rate=0.001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
# Train the model.
callbacks = myCallback()
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 40,
            epochs = 50,
            verbose = 2,
            callbacks = [callbacks])

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


plot_hist(hist)

In [ ]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )


unfreeze_model(model)

epochs = 4  # @param {type: "slider", min:4, max:10}
hist = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)
plot_hist(hist)

### **UNET**

### **SAVE MODEL**

In [ ]:
model_save_path = "./tmp/dummymodel.h5"
modelin.save(model_save_path)

In [ ]:
files.download('/content/tmp/dummymodel.h5')

In [ ]:
!pip install tensorflowjs

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import time
!tensorflowjs_converter \
    --input_format=keras dummymodel.h5 ./
# !tensorflowjs_converter --input_format=keras dummymodel.h5 dummymodel.json

In [ ]:
!zip dummypol.zip *.bin modelin.json